# **Laboratorio 9: Airflow 🛫**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Francisco Márquez
- Santiago Haberle

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/fco-marquez/MDS7202_Free_Riders)

## Temas a tratar

- Construcción de pipelines productivos usando `Airflow`.

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Reconocer los componentes pricipales de `Airflow` y su funcionamiento.
- Poner en práctica la construcción de pipelines de `Airflow`.
- Automatizar procesos típicos de un proyecto de ciencia de datos mediante `Airflow` y `Docker`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.tenor.com/OBQ6niqbxswAAAAM/legallyblonde.gif" width="300">
</p>

Vale, una estudiante del Magíster en Ciencia de Datos, se encuentra en la etapa final de sus estudios. Por un lado, está muy contenta por haber llegado tan lejos, pero por otro, no puede evitar sentirse inquieta. Desde que ingresó a la universidad, una pregunta la ha perseguido: ¿qué tan probable es que pueda ser seleccionada en los lugares donde envíe postulaciones para puestos de trabajo?

Esta duda la mantiene en constante reflexión, especialmente porque sabe que el mercado laboral en Ciencia de Datos es competitivo y exige habilidades no solo técnicas, sino también estratégicas para destacar. Sin embargo, Vale actualmente está completamente enfocada en terminar su tesis de magíster y ha tenido que postergar cualquier preparación específica para enfrentar el desafío de las postulaciones laborales.

Al ver el avance y las habilidades que usted ha demostrado en el curso, Vale decidió proponerle un desafío que le permitirá disminuir la incertidumbre sobre su futuro laboral. Inspirado en sus conocimientos, recolectó un conjunto de datos que contiene información sobre diversos factores que influyen en las decisiones de contratación de empresas al seleccionar entre sus postulantes. Este set de datos incluye los siguientes atributos:

- Age: Edad del candidato
- Gender: Genero del candidato. Male (0), Female (1).
- EducationLevel: Mayor nivel educacional alcanzado por el candidato. Licenciatura Tipo 1 (1), Licenciatura Tipo 2 (2), Maestría (3), PhD. (4).
- ExperienceYears: Años de experiencia profesional.
- PreviousCompanies: Numero de compañías donde el candidato ha trabajado anteriormente.
- DistanceFromCompany: Distancia en kilometros entre la residencia del candidato y la compañía donde postula.
- InterviewScore: Puntaje obtenido en la entrevista por el candidato entre 0 a 100.
- SkillScore: Puntaje obtenido en evaluación de habilidades técnicas por el candidato, entre 0 a 100.
- PersonalityScore: Puntaje obtenido en pruebas de personalidad del candidato, entre 0 a 100.
- RecruitmentStrategy: Estrategia del equipo de reclutamiento. Agresiva (1), Moderada (2), Conservadora (3).

Variable a predecir:
- HiringDecision: Resultado de la postulación. No contratado (0), Contratado (1).

Su objetivo será ayudar a Vale a desarrollar un modelo que le permita predecir, basado en estos factores, si un postulante será contratado o no. Esta herramienta no solo le dará a Vale mayor claridad sobre el impacto de ciertos atributos en la decisión final de contratación, sino que también le permitirá aplicar sus conocimientos de Ciencia de Datos para resolver una pregunta que a muchos estudiantes como a ella les inquieta.

Como estudiante del curso Laboratorio de Programación Científica para Ciencia de Datos, deberá demostrar sus capacidades para preprocesar, analizar y modelar datos, brindándole a Vale una solución robusta y bien fundamentada para su problemática.

`Nota:` El siguiente [enlace](https://www.kaggle.com/datasets/rabieelkharoua/predicting-hiring-decisions-in-recruitment-data/data) contiene el set de datos original.

# **1. Pipeline de Predicción Lineal** (30 Puntos)

<p align="center">
  <img src="https://media.licdn.com/dms/image/v2/D4E22AQHZplrdPyKnvA/feedshare-shrink_2048_1536/feedshare-shrink_2048_1536/0/1713736729086?e=2147483647&v=beta&t=Tad2ulaWkhhDrPRN0PCdXrfuza60PjoJqgLborDyLao" width="500">
</p>

En esta sección buscaremos desplegar un producto utilizando un modelo de clasificación `Random Forest` para determinar **si una persona será contratada o no en un proceso de selección**. Para ello, comenzaremos preparando un pipeline lineal mediante `Airflow`.

## **1.1 Preparando el Pipeline** (15 puntos)

**Primero, asegúrese de tener creada las carpetas `dags`, `plugins` y `logs`**.

Comenzamos preparando un archivo llamado `hiring_functions.py`, el cual guardará en la carpeta `dags` y debe contener lo siguiente:

1. (3 puntos) Una función llamada `create_folders()` que cree una carpeta, la cual utilice la fecha de ejecución como nombre. Adicionalmente, dentro de esta carpeta debe crear las siguientes subcarpetas:
  - raw
  - splits
  - models

  `Hint`: Puede hacer uso de kwargs para obtener la fecha de ejecución mediante el DAG. El siguiente [Enlace](https://airflow.apache.org/docs/apache-airflow/stable/templates-ref.html) le puede ser útil.

2. (3 puntos) Una función llamada `split_data()` que lea el archivo `data_1.csv` de la carepta `raw` y a partir de este, aplique un *hold out*, generando un dataset de entrenamiento y uno de prueba. Luego debe guardar estos nuevos conjuntos de datos en la carpeta `splits`. `Nota:` Utilice un 20% para el conjunto de prueba, mantenga la proporción original en la variable objetivo y fije una semilla.

3. (8 puntos) Cree una función llamada `preprocess_and_train()` que:
  - Lea los set de entrenamiento y prueba de la carpeta `splits`.
  - Cree y aplique un `Pipeline` con una etapa de preprocesamiento. Utilice `ColumnTransformers` para aplicar las transformaciones que estime convenientes. Puede apoyarse del archivo `data_1_report.html` para justificar cualquier paso del preprocesamiento.
  
  - Añada una etapa de entrenamiento utilizando el modelo `RandomForest`.
  
  Esta función **debe crear un archivo `joblib` (análogo a `pickle`) con el pipeline entrenado** en la carepta `models`, además debe **imprimir** el accuracy en el conjunto de prueba y el f1-score de la clase positiva (contratado).
3. (1 punto) Incorpore la función `gradio_interface` en su script, modificando la ruta de acceso a su modelo, de forma que pueda leerlo desde la carepta `models`. Puede realizar las modificaciones que estime necesarias.

`NOTA:` Se permite la creación de funciones auxiliares si lo estiman conveniente.

In [ ]:
import os

import gradio as gr
import joblib
import pandas as pd
from airflow.models import Variable
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler


def create_folders(ds, **kwargs):
    base_path = f"/root/airflow/{ds}"
    os.makedirs(f"{base_path}/raw", exist_ok=True)
    os.makedirs(f"{base_path}/splits", exist_ok=True)
    os.makedirs(f"{base_path}/models", exist_ok=True)


def split_data(ds, **kwargs):
    base_path = f"/root/airflow/{ds}"
    # Leer archivo data_1.csv desde la carpeta raw
    df = pd.read_csv(f"{base_path}/raw/data_1.csv")
    # Dividir los datos en conjuntos de entrenamiento y prueba (80% - 20%)
    train_df, test_df = train_test_split(
        df, test_size=0.2, stratify=df["HiringDecision"], random_state=1892
    )
    # Guardar los conjuntos divididos en la carpeta splits
    train_df.to_csv(f"{base_path}/splits/train.csv", index=False)
    test_df.to_csv(f"{base_path}/splits/test.csv", index=False)


def preprocess_and_train(ds, **kwargs):
    base_path = f"/root/airflow/{ds}"
    # Leer los conjuntos de entrenamiento y prueba desde la carpeta splits
    train_df = pd.read_csv(f"{base_path}/splits/train.csv")
    test_df = pd.read_csv(f"{base_path}/splits/test.csv")

    # Separar características y etiquetas
    X_train = train_df.drop("HiringDecision", axis=1)
    y_train = train_df["HiringDecision"]
    X_test = test_df.drop("HiringDecision", axis=1)
    y_test = test_df["HiringDecision"]

    # Pipeline de preprocesamiento
    numeric_features = [
        "Age",
        "DistanceFromCompany",
        "InterviewScore",
        "SkillScore",
        "PersonalityScore",
    ]
    ordinal_features = ["EducationLevel", "ExperienceYears"]

    categorical_features = ["Gender", "RecruitmentStrategy", "PreviousCompanies"]

    # Pipeline de preprocesamiento
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("ord", OrdinalEncoder(), ordinal_features),
            (
                "cat",
                OneHotEncoder(drop="first", sparse_output=False),
                categorical_features,
            ),
        ]
    )

    # Pipeline completo: preprocesamiento + modelo
    pipeline = Pipeline(
        [
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(random_state=1892, n_estimators=100)),
        ]
    )

    # Entrenar el pipeline completo
    pipeline.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = pipeline.predict(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, pos_label=1)  # Clase positiva = 1 (Contratado)

    print(f"Model Accuracy: {accuracy:.4f}")
    print(f"F1-Score (clase positiva - Contratado): {f1:.4f}")

    # Guardar el pipeline entrenado como archivo joblib
    model_path = f"/root/airflow/{ds}/models/hiring_model.joblib"
    joblib.dump(pipeline, model_path)
    print(f"Modelo guardado en: {model_path}")


In [ ]:
def predict(file, model_path):

    pipeline = joblib.load(model_path)
    input_data = pd.read_json(file)
    predictions = pipeline.predict(input_data)
    print(f"La prediccion es: {predictions}")
    labels = ["No contratado" if pred == 0 else "Contratado" for pred in predictions]

    return {"Predicción": labels[0]}


def gradio_interface(ds, **kwargs):

    model_path = f"/root/airflow/{ds}/models/hiring_model.joblib"

    interface = gr.Interface(
        fn=lambda file: predict(file, model_path),
        inputs=gr.File(label="Sube un archivo JSON"),
        outputs="json",
        title="Hiring Decision Prediction",
        description="Sube un archivo JSON con las características de entrada para predecir si Vale será contratada o no.",
    )
    interface.launch(share=True, server_name="0.0.0.0", server_port=7860)

## **1.2 Creando Nuestro DAG** (15 puntos)

<p align="center">
  <img src="https://media.tenor.com/a_yibuZQgngAAAAM/elle-woods.gif" width="400">
</p>

Con las funciones del pipeline ya creadas, ahora vamos a proceder a crear un Directed Acyclic Graph (DAG). Para ello, se le pide lo siguiente:

- (10 puntos) Cree un segundo archivo llamado `dag_lineal.py` y guardelo en la carpeta dags. Este script debe seguir la siguiente estructura (Ver imagen de referencia):

    0. Inicialice un DAG con fecha de inicio el 1 de octubre de 2024, ejecución manual y **sin backfill**. Asigne un `dag_id` que pueda reconocer facilmente, como `hiring_lineal`, etc.
    1. Debe comenzar con un marcador de posición que indique el inicio del pipeline.
    2. Cree una carpeta correspondiente a la ejecución del pipeline y cree las subcarpetas `raw`, `splits` y `models` mediante la función `create_folders()`.
    3. Debe descargar el archivo `data_1.csv` del siguiente [enlace](https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv). Debe guardar el archivo en la carpeta raw de la ejecución correspondiente.`Hint:` Le puede ser útil el comando `curl -o <path de guardado> <enlace con los datos>`.
    4. Debe aplicar un hold out mediante la función `split_data()` de su archivo creado en la subsección anterior.
    5. Debe aplicar el preprocesamiento y el entrenamiento del modelo mediante la función `preprocess_and_train()`.
    6. Finalmente, debe montar una interfaz en gradio donde pueda cargar un archivo ``json``.


- (3 puntos) Cree un `DockerFile` para montar un contenedor que contenga Airflow. Adicionalmente, cree una carpeta llamada dags donde guardará el script.py creado anteriormente.

    `Nota:` Para la imagen, se recomienda utilizar python 3.10-slim. Adicionalmente, puede instalar `curl` mediante la siguiente linea de código: `RUN apt-get update && apt-get install -y curl`.

- Construya el contenedor en Docker y acceda a la aplicación web de Airflow mediante el siguiente [enlace](http://localhost:8080/). Inicie sesión, acceda al DAG creado y ejecute de forma manual su pipeline.

- (2 puntos) Acceda a la URL pública de Gradio e ingrese el archivo `vale_data.json` a su modelo. ¿Que predicción entregó el modelo para Vale? Adjunte imágenes de su resultado. `Hint:` Puede acceder a los `logs` para obtener los prints y la URL pública.

`Hint:` Recuerde que puede entregar `kwargs` a sus funciones, como por ejemplo la fecha de ejecución `ds`.

**Para esta sección, debe adjuntar todos los scripts creados junto a su notebook en la entrega, ya que serán ejecutados para validar el funcionamiento. Para justificar sus respuestas, adicionaslmente puede utilizar imágenes de apoyo, como screenshots.**

DAG de referencia:
<p align="center">
  <img src="https://drive.google.com/uc?id=1iwDgECZfFeWq1dl433tMa6_3CNF9cn1L" width="1200">
</p>




In [ ]:
import datetime as dt

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.empty import EmptyOperator
from airflow.operators.python import PythonOperator
from airflow.utils.dates import days_ago
from hiring_functions import (
    create_folders,
    gradio_interface,
    predict,
    preprocess_and_train,
    split_data,
)

default_args = {
    "owner": "free-riders",
    "retries": 1,
}

with DAG(
    dag_id="hiring_lineal",
    default_args=default_args,
    description="Hiring Decision Workflow with Linear DAG",
    start_date=dt.datetime(2024, 10, 1),
    schedule_interval=None,
    catchup=False,
) as dag:

    # Task 1 - Just a simple print statement
    dummy_task = EmptyOperator(task_id="Starting_the_process", retries=2)

    # Task 2 - Create necessary folders
    create_folders_task = PythonOperator(
        task_id="create_folders",
        python_callable=create_folders,
        op_kwargs={"ds": "{{ ds }}"},
    )

    # Task 3 - Download data using BashOperator
    task_download_dataset_1 = BashOperator(
        task_id="download_dataset_1",
        bash_command="curl -o "
        "/root/airflow/{{ ds }}/raw/data_1.csv "
        "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv",
    )

    # Task 4 - Split data
    split_data_task = PythonOperator(
        task_id="split_data", python_callable=split_data, op_kwargs={"ds": "{{ ds }}"}
    )

    # Task 5 - Preprocess and train model
    preprocess_and_train_task = PythonOperator(
        task_id="preprocess_and_train",
        python_callable=preprocess_and_train,
        op_kwargs={"ds": "{{ ds }}"},
    )

    # Task 6 - Set up Gradio interface
    gradio_interface_task = PythonOperator(
        task_id="gradio_interface",
        python_callable=gradio_interface,
        op_kwargs={"ds": "{{ ds }}"},
    )

    # Define task dependencies
    (
        dummy_task
        >> create_folders_task
        >> task_download_dataset_1
        >> split_data_task
        >> preprocess_and_train_task
        >> gradio_interface_task
    )


#### Dockerfile

```dockerfile
# Utiliza una imagen base con Python instalado
FROM python:3.10-slim

# Establece el directorio de trabajo en el contenedor
WORKDIR /root/airflow

# Establece la variable de entorno AIRFLOW_HOME
ENV AIRFLOW_HOME=/root/airflow

# Instalar dependencias del sistema
RUN apt-get update && apt-get install -y \
    curl \
    && apt-get clean \
    && rm -rf /var/lib/apt/lists/*

# Actualizar pip
RUN python -m pip install --upgrade pip

# Instala Apache Airflow y otras librerías
COPY requirements.txt .
RUN pip install -r requirements.txt

# Inicializa la base de datos de Airflow
RUN airflow db migrate


# Crea el usuario admin de Airflow
RUN airflow users create --role Admin --username admin --email admin \
 --firstname admin --lastname admin --password admin

# Copia las carpetas necesarias al contenedor
COPY ./dags $AIRFLOW_HOME/dags
COPY ./logs $AIRFLOW_HOME/logs
COPY ./plugins $AIRFLOW_HOME/plugins

# Expone el puerto 8080 para el servidor web de Airflow
EXPOSE 8080 7860

# Comando para iniciar el servidor web y el scheduler
CMD ["sh", "-c", "airflow webserver -p 8080 & airflow scheduler"]
```

#### requirements.txt
```
apache-airflow==2.11.0
apache-airflow-providers-fab
scikit-learn
datetime
pandas
numpy
joblib
gradio
```

# **2. Paralelizando el Pipeline** (30 puntos)

<p align="center">
  <img src="https://i.gifer.com/8LNL.gif" width="400">
</p>



Al ver los resultados obtenidos, Vale queda muy contenta con el clasificador. Sin embargo, le aparecen algunas dudas respecto al funcionamiento del pipeline. Primero le comenta que es posible que en un futuro tenga nuevos datos que podrían ser útiles para realizar nuevos entrenamientos, por lo que sería ideal si este pipeline se fuera ejecutando de forma periódica y **NO** de forma manual. Además, Vale le menciona que le gustaría explorar el desempeño de otros modelos además de `Random Forest`, de forma que el pipeline seleccione de forma automática el modelo con mejor desempeño para luego hacer la predicción de Vale.

## **2.1 Preparando un Nuevo Pipeline** (15 puntos)

<p align="center">
  <img src="https://media.tenor.com/gnA7-5TewXMAAAAM/elle-woods.gif" width="400">
</p>

De acuerdo a lo que le comentó Vale, usted decide crear un nuevo script con las funciones que utilizará su pipeline. Por ende, dentro de la carpeta `dags`, usted creará el archivo `hiring_dynamic_functions.py` el cual debe contener:

1. (2 puntos) Una función llamada `create_folders()` que cree una carpeta, la cual utilice la fecha de ejecución como nombre. Adicionalmente, dentro de esta carpeta debe crear las siguientes subcarpetas:
  - raw
  - preprocessed
  - splits
  - models
2. (2 puntos) Una función llamada `load_ands_merge()` que lea desde la carpeta `raw` los archivos `data_1.csv`y `data_2.csv` en caso de estar disponible. Luego concatene estos y genere un nuevo archivo resultante, guardándolo en la carpeta `preprocessed`.

3. (2 puntos) Una función llamada `split_data()` que lea la data guardada en la carpeta `preprocessed` y realice un hold out sobre esta data. Esta función debe crear un conjunto de entrenamiento y uno de prueba. Mantenga una semilla y 20% para el conjunto de prueba. Guarde los conjuntos resultantes en la carpeta `splits`.

4. (6 puntos) Una función llamada `train_model()` que reciba un modelo de clasificación.
    - La función debe comenzar leyendo el conjunto de entrenamiento desde la carpeta `spits`.
    - Esta debe crear y aplicar un `Pipeline` con una etapa de preprocesamiento. Utilice `ColumnTransformers` para aplicar las transformaciones que estime convenientes.
    - Añada una etapa de entrenamiento utilizando un modelo que ingrese a la función.
  
  Esta función **debe crear un archivo joblib con el pipeline entrenado**. Guarde el modelo con un nombre que le permita una facil identificación dentro de la carpeta `models`.

5. (3 puntos) Una función llamada `evaluate_models()` que reciba sus modelos entrenados desde la carpeta `models`, evalúe su desempeño mediante `accuracy` en el conjunto de prueba y seleccione el mejor modelo obtenido. Luego guarde el mejor modelo como archivo `.joblib`. Su función debe imprimir el nombre del modelo seleccionado y el accuracy obtenido.

In [ ]:
import os

import gradio as gr
import joblib
import pandas as pd
from airflow.models import Variable
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler


def create_folders(ds, **kwargs):
    base_path = f"/root/airflow/{ds}"
    os.makedirs(f"{base_path}/raw", exist_ok=True)
    os.makedirs(f"{base_path}/preprocessed", exist_ok=True)
    os.makedirs(f"{base_path}/splits", exist_ok=True)
    os.makedirs(f"{base_path}/models", exist_ok=True)


def load_ands_merge(ds, **kwargs):
    base_path = f"/root/airflow/{ds}"
    # Leer archivo data_1.csv desde la carpeta raw
    df1 = pd.read_csv(f"{base_path}/raw/data_1.csv")
    # Leer archivo data_2.csv desde la carpeta raw si es que esta disponible
    df2 = (
        pd.read_csv(f"{base_path}/raw/data_2.csv")
        if os.path.exists(f"{base_path}/raw/data_2.csv")
        else pd.DataFrame()
    )
    # concatenar los dataframes
    df = pd.concat([df1, df2], ignore_index=True)
    # Guardar el dataframe combinado
    df.to_csv(f"{base_path}/preprocessed/combined_data.csv", index=False)


def split_data(ds, **kwargs):
    base_path = f"/root/airflow/{ds}"
    df = pd.read_csv(f"{base_path}/preprocessed/combined_data.csv")
    train_df, test_df = train_test_split(
        df, test_size=0.2, stratify=df["HiringDecision"], random_state=1892
    )
    # Guardar los conjuntos divididos en la carpeta splits
    train_df.to_csv(f"{base_path}/splits/train.csv", index=False)
    test_df.to_csv(f"{base_path}/splits/test.csv", index=False)


def train_model(ds, model, **kwargs):
    base_path = f"/root/airflow/{ds}"
    # Leer los conjuntos de entrenamiento y prueba desde la carpeta splits
    train_df = pd.read_csv(f"{base_path}/splits/train.csv")
    test_df = pd.read_csv(f"{base_path}/splits/test.csv")

    # Separar características y etiquetas
    X_train = train_df.drop("HiringDecision", axis=1)
    y_train = train_df["HiringDecision"]
    X_test = test_df.drop("HiringDecision", axis=1)
    y_test = test_df["HiringDecision"]

    # Pipeline de preprocesamiento
    numeric_features = [
        "Age",
        "DistanceFromCompany",
        "InterviewScore",
        "SkillScore",
        "PersonalityScore",
    ]
    ordinal_features = ["EducationLevel", "ExperienceYears"]

    categorical_features = ["Gender", "RecruitmentStrategy", "PreviousCompanies"]

    # Pipeline de preprocesamiento
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("ord", OrdinalEncoder(), ordinal_features),
            (
                "cat",
                OneHotEncoder(drop="first", sparse_output=False),
                categorical_features,
            ),
        ]
    )
    # Pipeline completo: preprocesamiento + modelo
    pipeline = Pipeline(
        [
            ("preprocessor", preprocessor),
            ("classifier", model),
        ]
    )

    pipeline.fit(X_train, y_train)

    # Guardar el modelo entrenado
    joblib.dump(pipeline, f"{base_path}/models/hiring_{model.__class__.__name__}.pkl")


def evaluate_models(ds, **kwargs):
    base_path = f"/root/airflow/{ds}"
    test_df = pd.read_csv(f"{base_path}/splits/test.csv")
    X_test = test_df.drop("HiringDecision", axis=1)
    y_test = test_df["HiringDecision"]

    best_model = None
    best_accuracy = 0.0
    best_model_name = ""

    for model_file in os.listdir(f"{base_path}/models"):
        if model_file.endswith(".pkl"):
            model_path = os.path.join(f"{base_path}/models", model_file)
            model = joblib.load(model_path)
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Model: {model_file}, Accuracy: {accuracy:.4f}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model
                best_model_name = model_file.strip(".pkl")

    if best_model is not None:
        joblib.dump(best_model, f"{base_path}/models/best_hiring_model.pkl")
        print(f"Best Model: {best_model_name} with Accuracy: {best_accuracy:.4f}")


## **2.2 Componiendo un nuevo DAG** (15 puntos)

<p align="center">
  <img src="https://67.media.tumblr.com/bfa5208006dc3f404ec08e8c3195cf2c/tumblr_obg9tgnLfX1u9e9f2o2_r1_500.gif" width="500">
</p>

Con las nuevas funciones, se debe crear el nuevo nuevo DAG. Para ello, cree un nuevo script en la carpeta `dags`, llamandolo `dag_dynamic.py`. Este script debe contener la siguiente estructura:

1. (1 punto) Inicialice un DAG con fecha de inicio el 1 de octubre de 2024, el cual se debe ejecutar el día 5 de cada mes a las 15:00 UTC. Utilice un `dag_id` interpretable para identificar fácilmente. **Habilite el backfill** para que pueda ejecutar tareas programadas desde fechas pasadas.
2. (1 punto) Comience con un marcador de posición que indique el inicio del pipeline.
3. (2 puntos) Cree una carpeta correspondiente a la ejecución del pipeline y cree las subcarpetas `raw`, `preprocessed`, `splits` y `models` mediante la función `create_folders()`.
4. (2 puntos) Implemente un `Branching`que siga la siguiente lógica:
  - Fechas previas al 1 de noviembre de 2024: Se descarga solo `data_1.csv`
  - Desde el 1 de noviembre del 2024: descarga `data_1.csv` y `data_2.csv`.
  En el siguiente [enlace](https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_2.csv) puede descargar `data_2.csv`.
5. (1 punto) Cree una tarea que concatene los datasets disponibles mediante la función `load_and_merge()`. Configure un `Trigger` para que la tarea se ejecute si encuentra disponible **como mínimo** uno de los archivos.
6. (1 punto) Aplique el hold out al dataset mediante la función `split_data()`, obteniendo un conjunto de entrenamiento y uno de prueba.
7. (2 puntos) Realice 3 entrenamientos en paralelo:
  - Un modelo Random Forest.
  - 2 modelos a elección.
  Asegúrese de guardar sus modelos entrenados con nombres distintivos. Utilice su función `train_model()` para ello.
8. (2 puntos) Mediante la función `evaluate_models()`, evalúe los modelos entrenados, registrando el accuracy de cada modelo en el set de prueba. Luego debe imprimir el mejor modelo seleccionado y su respectiva métrica. Configure un `Trigger` para que la tarea se ejecute solamente si los 3 modelos fueron entrenados y guardados.

`Hint:` Recuerde que puede entregar `kwargs` a sus funciones, como por ejemplo la fecha de ejecución `ds`.

Una vez creado el script, vuelva a construir el contenedor en Docker, acceda a la aplicación web de Airflow, ejecute su pipeline y muestre sus resultados. Adjunte imágenes que ayuden a mostrar el proceso y sus resultados.

Adicionalmente, responda (1 c/u):

- ¿Cual es el accuracy de cada modelo en la ejecución de octubre? ¿Se obtienen los mismos resultados a partir de Noviembre?
- Analice como afectó el añadir datos a sus modelos mediante el desempeño del modelo y en costo computacional.
- Muestre el esquema de su DAG ejecutado en octubre y en noviembre.


`Nota:` Para esta sección no debe implementar la tarea en gradio, solamente se espera determinar el mejor modelo y comparar el desempeño obtenido.

**IMPORTANTE: Para esta sección, debe adjuntar todos los scripts creados junto a su notebook en la entrega, ya que serán ejecutados para validar el funcionamiento. Para justificar sus respuestas, adicionaslmente puede utilizar imágenes de apoyo, como screenshots.**

In [ ]:
import datetime as dt

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.empty import EmptyOperator
from airflow.operators.python import BranchPythonOperator, PythonOperator
from airflow.utils.dates import days_ago
from hiring_dynamic_functions import (
    create_folders,
    evaluate_models,
    load_and_merge,
    split_data,
    train_model,
)
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression

default_args = {
    "owner": "free-riders",
    "retries": 1,
}

with DAG(
    dag_id="hiring_dynamic",
    default_args=default_args,
    description="Hiring Decision Workflow with Dynamic Tasks",
    start_date=dt.datetime(2024, 10, 1),
    schedule_interval="0 15 5 * *",  # cada dia 5 del mes a las 15:00 UCT
    catchup=True,
) as dag:

    # Task 1 - Inicial dummy task
    dummy_task = EmptyOperator(task_id="Starting_the_process", retries=2)

    # Task 2 - Crear las carpetas necesarias
    create_folders_task = PythonOperator(
        task_id="create_folders",
        python_callable=create_folders,
        op_kwargs={"ds": "{{ ds }}"},
    )

    # Branching: Antes del 1-11-2024 descargar solo data_1.csv, despues ambas
    branch_task = BranchPythonOperator(
        task_id="branch_task",
        python_callable=lambda **kwargs: (
            "download_dataset_1"
            if kwargs['execution_date'] < dt.datetime(2024, 11, 1, tzinfo=kwargs['execution_date'].tzinfo)
            else ["download_dataset_1", "download_dataset_2"]
        ),
        provide_context=True,
    )

    # Task 3a - Descargar dataset 1
    task_download_dataset_1 = BashOperator(
        task_id="download_dataset_1",
        bash_command="curl -o "
        "/root/airflow/{{ ds }}/raw/data_1.csv "
        "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv",
    )

    # Task 3b - Descargar dataset 2
    task_download_dataset_2 = BashOperator(
        task_id="download_dataset_2",
        bash_command="curl -o "
        "/root/airflow/{{ ds }}/raw/data_2.csv "
        "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_2.csv",
    )

    # Task 4 - Load and merge data con Trigger  para ejecutar si encuentra como minimo un dataset
    load_and_merge_task = PythonOperator(
        task_id="load_and_merge",
        python_callable=load_and_merge,
        op_kwargs={"ds": "{{ ds }}"},
        trigger_rule="one_success",
    )

    # Task 5 - Split data
    split_data_task = PythonOperator(
        task_id="split_data", python_callable=split_data, op_kwargs={"ds": "{{ ds }}"}
    )

    # Task 6 - Train models in parallel
    models = [
        RandomForestClassifier(random_state=1892, n_estimators=100),
        GradientBoostingClassifier(random_state=1892),
        LogisticRegression(max_iter=200),
    ]
    train_model_tasks = []
    for i, model in enumerate(models):
        task = PythonOperator(
            task_id=f"train_model_{i+1}",
            python_callable=train_model,
            op_kwargs={"ds": "{{ ds }}", "model": model},
        )
        train_model_tasks.append(task)

    # Task 7 - Evaluate models
    evaluate_models_task = PythonOperator(
        task_id="evaluate_models",
        python_callable=evaluate_models,
        op_kwargs={"ds": "{{ ds }}"},
        trigger_rule="all_success",
    )

    # Setting up dependencies
    (
        dummy_task
        >> create_folders_task
        >> branch_task
        >> [task_download_dataset_1, task_download_dataset_2]
        >> load_and_merge_task
        >> split_data_task
        >> train_model_tasks
        >> evaluate_models_task
    )

### Resultados de Ejecución

#### 1. ¿Cual es el accuracy de cada modelo en la ejecución de octubre? ¿Se obtienen los mismos resultados a partir de Noviembre?

**Ejecución de Octubre (2024-10-05 - solo data_1.csv):**

![Accuracy Octubre](imagenes/accuracy_october.png)

Los resultados muestran que en la ejecución de octubre, donde el pipeline solo utilizó el archivo data_1.csv, el modelo RandomForestClassifier obtuvo un accuracy de 0.8867, mientras que GradientBoostingClassifier alcanzó 0.8933, convirtiéndose en el mejor modelo de esta ejecución. Por su parte, LogisticRegression presentó el desempeño más bajo con 0.8600.

**Ejecución de Noviembre (2024-11-05 - data_1.csv + data_2.csv):**

![Accuracy Noviembre](imagenes/accuracy_november.png)

En contraste, la ejecución de noviembre, que incorporó tanto data_1.csv como data_2.csv, mostró resultados notablemente diferentes. RandomForestClassifier alcanzó 0.9300, GradientBoostingClassifier obtuvo 0.9233, y LogisticRegression mejoró levemente a 0.8667. Es importante destacar que el mejor modelo cambió de GradientBoosting en octubre a RandomForest en noviembre.

**Análisis:**

Los resultados claramente demuestran que no se obtienen los mismos valores entre ambas ejecuciones. La incorporación de datos adicionales generó una mejora generalizada en todos los modelos, siendo RandomForest el que experimentó el incremento más significativo con 4.33 puntos porcentuales. Este comportamiento sugiere que RandomForest posee una mayor capacidad para aprovechar conjuntos de datos más extensos, permitiéndole capturar patrones más complejos y diversos. GradientBoosting también mostró una mejora considerable de 3 puntos porcentuales, mientras que LogisticRegression presentó el menor incremento con apenas 0.67 puntos, lo que podría indicar que los modelos lineales tienen limitaciones para beneficiarse de datos adicionales cuando existen relaciones no lineales en los datos.

#### 2. Analice como afectó el añadir datos a sus modelos mediante el desempeño del modelo y en costo computacional

**Desempeño del Modelo:**

La incorporación de datos adicionales tuvo un impacto significativo en el desempeño de los modelos evaluados. Al comparar los resultados de octubre y noviembre, se observa que todos los clasificadores experimentaron mejoras en sus métricas de accuracy, aunque en magnitudes diferentes. RandomForest demostró ser el modelo que más se benefició de la ampliación del conjunto de datos, pasando de 0.8867 a 0.9300, lo que representa un incremento de 4.33 puntos porcentuales. Esta mejora no solo fue la más pronunciada, sino que también permitió que RandomForest superara a GradientBoosting como el mejor modelo del pipeline. Por su parte, GradientBoosting también mostró una mejora sustancial de 3 puntos porcentuales, mientras que LogisticRegression presentó el menor incremento con apenas 0.67 puntos.

Este comportamiento diferenciado entre los modelos puede atribuirse a sus características intrínsecas. Los modelos basados en ensambles de árboles, como RandomForest y GradientBoosting, poseen una mayor capacidad para capturar relaciones complejas y no lineales en los datos, lo que les permite aprovechar mejor conjuntos de datos más extensos. En contraste, LogisticRegression, al ser un modelo lineal, muestra limitaciones inherentes para beneficiarse de datos adicionales cuando los patrones subyacentes son predominantemente no lineales. El incremento de datos permitió una mejor generalización de los modelos, alcanzando el mejor resultado un accuracy de 93%, lo que representa una mejora notable respecto al 89% obtenido con el conjunto de datos más reducido.

**Costo Computacional:**

El análisis del costo computacional revela información interesante sobre la escalabilidad del pipeline implementado. La ejecución completa del DAG en octubre tomó 4 minutos y 15 segundos, mientras que en noviembre el tiempo se extendió a 5 minutos y 18 segundos. Este incremento incluye no solo el tiempo adicional de entrenamiento de los modelos con más datos, sino también las operaciones de descarga del segundo archivo, el merge de ambos datasets, y el preprocesamiento de un volumen mayor de información.

Lo destacable de estos resultados es que, a pesar de aproximadamente duplicar el tamaño del dataset de entrenamiento, el tiempo de ejecución solo aumentó en aproximadamente un cuarto. Esto sugiere que el pipeline escala de manera eficiente y que la arquitectura implementada maneja adecuadamente incrementos en el volumen de datos.

**Trade-off y Conclusión:**

Al evaluar la relación entre costo y beneficio, queda claro que la incorporación de datos adicionales representa una decisión altamente favorable desde el punto de vista del rendimiento del pipeline. Por un lado, se obtuvo una ganancia en accuracy para el mejor modelo, elevando la precisión de 88.67% a 93%. Por otro lado, el costo asociado fue únicamente un minuto adicional de procesamiento, lo que en términos prácticos resulta despreciable para la mayoría de aplicaciones productivas. La relación costo-beneficio es particularmente atractiva: por cada minuto extra de ejecución, se obtiene una mejora significativa en la calidad de las predicciones. En un entorno productivo donde la precisión del modelo puede traducirse directamente en mejor toma de decisiones para Vale, este incremento marginal en tiempo de procesamiento es completamente justificable y representa una inversión eficiente de recursos computacionales.

#### 3. Muestre el esquema de su DAG ejecutado en octubre y en noviembre

**Esquema del DAG - Ejecución de Octubre:**

![DAG Graph Octubre](imagenes/graph_october.png)

En la ejecución de octubre, el branching implementado en el DAG evalúa la fecha de ejecución y determina que, al ser anterior al 1 de noviembre de 2024, solo debe descargar el archivo data_1.csv. Esto se refleja en el grafo donde únicamente la tarea `download_dataset_1` se ejecuta, mientras que `download_dataset_2` permanece en estado omitido (skipped). El flujo continúa normalmente con las tareas de merge, split, entrenamiento paralelo de los tres modelos y evaluación final.

**Esquema del DAG - Ejecución de Noviembre:**

![DAG Graph Noviembre](imagenes/graph_november.png)

Por el contrario, en la ejecución de noviembre, el mecanismo de branching detecta que la fecha de ejecución es posterior al 1 de noviembre de 2024, por lo que decide ejecutar ambas tareas de descarga: `download_dataset_1` y `download_dataset_2`. Esta diferencia es crucial y se aprecia claramente en el grafo, donde ambas tareas aparecen en estado de ejecución exitosa. Posteriormente, la tarea de merge combina ambos archivos antes de proceder con el resto del pipeline.

**Análisis:**

La diferencia fundamental entre ambas ejecuciones reside en el comportamiento del operador `BranchPythonOperator` implementado en la tarea `branch_task`. Este operador evalúa dinámicamente la fecha de ejecución mediante `kwargs['execution_date']` y compara esta fecha con el umbral establecido del 1 de noviembre de 2024. Para fechas anteriores, retorna únicamente el identificador de la tarea `download_dataset_1`, mientras que para fechas posteriores retorna una lista conteniendo ambos identificadores de descarga. Este mecanismo permite que el pipeline se adapte automáticamente al contexto temporal, descargando solo los datos disponibles en cada momento sin requerir intervención manual.

Es importante destacar que, independientemente de la rama tomada en el branching, el resto del pipeline permanece estructuralmente idéntico. La tarea `load_and_merge` está configurada con `trigger_rule="one_success"`, lo que garantiza su ejecución siempre que al menos una de las tareas de descarga se complete exitosamente. Esto proporciona robustez al pipeline, asegurando que pueda continuar operando incluso si solo uno de los archivos está disponible. Posteriormente, las tareas de split, entrenamiento paralelo y evaluación se ejecutan de manera idéntica en ambos escenarios, procesando el dataset resultante independientemente de si proviene de uno o dos archivos fuente. Esta arquitectura flexible y resiliente demuestra las capacidades de Airflow para implementar lógica condicional compleja manteniendo la claridad y mantenibilidad del código.

# Conclusión

Éxito!
<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1000/1*PX8WVijZapo7EDrvGv9Inw.gif" width="500">
</div>
